# Neural Network using tensorflow
for this model whe used some information from "https://machinelearningmastery.com/tensorflow-tutorial-deep-learning-with-tf-keras/"

In [1]:
# import libraries
import tensorflow.keras as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import keras_tuner as kt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Activation, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam

## Load the data


In [2]:
"""
# load complete data
complete_df = pd.read_csv('data/AmesHousingPreprocessed.csv')
complete_df = complete_df.drop('Unnamed: 0', axis=1)

# split data into input and target
target = complete_df['SalePrice']
input_df = complete_df.drop('SalePrice', axis=1)

display(input_df.head())

# convert the pandas dataframes to numpy ndarrays
X_np = input_df.to_numpy()
y_np = target.to_numpy()

# split the data into 70% training and 30% testing
X_train, X_test, y_train, y_test = train_test_split(X_np, y_np, train_size=0.7, random_state=1265599650)
"""

"\n# load complete data\ncomplete_df = pd.read_csv('data/AmesHousingPreprocessed.csv')\ncomplete_df = complete_df.drop('Unnamed: 0', axis=1)\n\n# split data into input and target\ntarget = complete_df['SalePrice']\ninput_df = complete_df.drop('SalePrice', axis=1)\n\ndisplay(input_df.head())\n\n# convert the pandas dataframes to numpy ndarrays\nX_np = input_df.to_numpy()\ny_np = target.to_numpy()\n\n# split the data into 70% training and 30% testing\nX_train, X_test, y_train, y_test = train_test_split(X_np, y_np, train_size=0.7, random_state=1265599650)\n"

In [4]:
# work with given training and testing data from kaggle competition
train_data = pd.read_csv('data/train_clean.csv')
train_target = train_data['SalePrice']
train_input = train_data.drop('SalePrice', axis=1)

test_data = pd.read_csv('data/test_clean.csv')
test_target = test_data['SalePrice']
test_input = test_data.drop('SalePrice', axis=1)


X_train = train_input.to_numpy()
y_train = train_target.to_numpy()
X_test = test_input.to_numpy()
y_test = test_target.to_numpy()

FileNotFoundError: [Errno 2] No such file or directory: 'data/clean_data.csv'

In [ ]:
# find number of features
n_features = input_df.shape[1]

## Build the model
The code that is written computes its own neural network models and picks the best out of it. We continue with that model.

In [ ]:
def build_model(hp):
    """
    This function computes a/the best neural network for the given data. 
    It makes a model by tuning the layers and parameters of the layers for 
    the amount of trials given in the tuner variable.
    
    source: "https://keras.io/guides/keras_tuner/getting_started/"
    """
    
    # metrics for the layers
    m1 = tf.metrics.RootMeanSquaredError()
    m2 = 'mean_absolute_percentage_error'
    
    
    '''
    #define parameters
    start_neurons=hp.Int(name='start_neurons', min_value=32, max_value=512, step=32),
    net_depth = hp.Int(name='start_depth', min_value=2, max_value=6)
    dropout=hp.Boolean(name = 'dropout', default=False),
    bn_after_act=hp.Boolean(name = 'bn_after_act', default=False),
    activation=hp.Choice("activation", ["relu", "leaky_relu", "elu", "tanh"])
    
    input_size =(n_features)
    
    model = u(input_size = input_size,
              start_neurons = start_neurons,
              net_depth = net_depth,
              dropout = dropout,
              bn_after_act = bn_after_act,
              activation = activation)
    '''
    # compute a model
    model = Sequential()
        
      
    # tune number of layers
    for i in range(hp.Int("numlayers", 1, 4)):
        
        model.add(
            
            # create dense layer
            Dense(
                
                # Tune number of units separately.
                units=hp.Int(f"units{i}", min_value=16, max_value=256, step=16),
                activation=hp.Choice("activation", ["relu", "leaky_relu", "elu", "tanh"]),
            ))
        
        # check if drop-out layer is benneficial 
        if hp.Boolean("dropout"):
            model.add(Dropout(rate=0.3))
        
        # check if batch normalization is benneficial
        if hp.Boolean("bn_after_act"):
            model.add(BatchNormalization())
      
     
    # output layer
    model.add(Dense(1))
    
    model.compile(optimizer='Adam', loss=tf.metrics.mean_squared_error, metrics=[m1, m2])
    
    return model

tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_loss",
    max_trials=50,
    executions_per_trial=2,
    overwrite=True,
    directory="data",
    project_name="NN1",
)

# train the models with the given data
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))
best_model = tuner.get_best_models()[0]

In [ ]:
# fit the best model
best_model.fit(X_train, y_train,
          batch_size=32, epochs=200,
          validation_data=(X_test, y_test))



## Visualizing the results
Now that the model is actually build, we can visualize the outcome.

In [ ]:
best_model.summary()

In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.results_summary()

In [ ]:
loss_df = pd.DataFrame(best_model.history.history)

loss_df.plot(figsize=(12,8))
plt.title("Model information")
plt.xlabel("epochs")
plt.show()

loss_df.plot(figsize=(12,8))
plt.ylim((0,0.02))
plt.title("Model information")
plt.xlabel("epochs")
plt.show()

In [ ]:
from sklearn import metrics

y_pred = best_model.predict(X_test)
var_score = metrics.explained_variance_score(y_test,y_pred)
# compute the accuracy of the model 
print('Variance score:', var_score)
print('\nRMSE:',loss_df['root_mean_squared_error'].tail(1))
print('\nLoss:',loss_df['loss'].tail(1))
print('\nMAPE:',loss_df['mean_absolute_percentage_error'].tail(1))

In [ ]:
print(y_pred[0], y_test[0])

### Model 2

In [ ]:
#### verbeterpunten:
# validation data gebruiken                           
# k folds
# andere metrics voor aflezen                     check
# adam vs andere andere sgd methods testen        check, Adam ziet er het best uit
# hidden layers toevoegen 
# dropout layers toevoegen                        dropout layers hebben geen positief effect op het model
# aantal nodes per layer aanpassen
# batchsize aanpassen (32, 64 of 128)             check, 32 lijkt het beste te werken
# aantal epochs                                   check, na 20 epochs lijkt het model het meest te leren, daarna improved het niet heel veel meer                                     
# onze loss plotten                               check
